Gabriel Avinaz\
Week 7\
4/26/23

## Assignment 7.1
### Part a

In [1]:
import os
import shutil
import pandas as pd

In [2]:
path = "data"
if not os.path.exists(path):
    os.makedirs(path)
    shutil.copy("../assignment03/results/routes.parquet", path)

In [3]:
routes_df = pd.read_parquet("data/routes.parquet", engine='pyarrow')
routes_df = pd.json_normalize(routes_df["Flight Info"])
routes_df.sample(10)

,codeshare,equipment,airline.active,airline.airline_id,airline.alias,airline.callsign,airline.country,airline.iata,airline.icao,airline.name,...,src_airport.icao,src_airport.latitude,src_airport.longitude,src_airport.name,src_airport.source,src_airport.timezone,src_airport.type,src_airport.tz_id,dst_airport,src_airport
18187,False,"[320, E90]",True,1767,SN Brussels Airlines,CHINA SOUTHERN,China,CZ,CSN,China Southern Airlines,...,ZGGG,23.392401,113.299004,Guangzhou Baiyun International Airport,OurAirports,8.0,airport,Asia/Shanghai,NaN,NaN
14722,False,"[320, 767]",True,1355,Pulkovo Aviation Enterprise,SPEEDBIRD,United Kingdom,BA,BAW,British Airways,...,EGLL,51.470600,-0.461941,London Heathrow Airport,OurAirports,0.0,airport,Europe/London,NaN,NaN
45345,False,[DH1],True,-1,\N,\N,\N,-,nan,Unknown,...,CYWK,52.921902,-66.864403,Wabush Airport,OurAirports,-4.0,airport,America/Halifax,NaN,NaN
13988,False,[77W],True,3029,nan,JETBLUE,United States,B6,JBU,JetBlue Airways,...,OMDB,25.252800,55.364399,Dubai International Airport,OurAirports,4.0,airport,Asia/Dubai,NaN,NaN
15003,False,[737],True,4740,Swiss European,SKYMARK,Japan,BC,SKY,Skymark Airlines,...,RJAA,35.764702,140.386002,Narita International Airport,OurAirports,9.0,airport,Asia/Tokyo,NaN,NaN
40351,False,[737],True,5484,Varig,XIAMEN AIR,China,MF,CXA,Xiamen Airlines,...,ZGSZ,22.639299,113.810997,Shenzhen Bao'an International Airport,OurAirports,8.0,airport,Asia/Shanghai,NaN,NaN
59592,False,"[757, 763]",True,5265,TWA,U S AIR,United States,US,USA,US Airways,...,SPIM,-12.021900,-77.114305,Jorge Chávez International Airport,OurAirports,-5.0,airport,America/Lima,NaN,NaN
24070,True,[330],True,2222,Emirates Airlines,ETIHAD,United Arab Emirates,EY,ETD,Etihad Airways,...,OMAA,24.433001,54.651100,Abu Dhabi International Airport,OurAirports,4.0,airport,Asia/Dubai,NaN,NaN
21260,True,"[CRJ, ERJ]",True,2009,CSA Czech Airlines,DELTA,United States,DL,DAL,Delta Air Lines,...,KMEM,35.042400,-89.976700,Memphis International Airport,OurAirports,-6.0,airport,America/Chicago,NaN,NaN
66593,False,[AT7],True,-1,\N,\N,\N,-,nan,Unknown,...,HTDA,-6.878110,39.202599,Julius Nyerere International Airport,OurAirports,3.0,airport,Africa/Dar_es_Salaam,NaN,NaN


In [4]:
routes_df = routes_df.dropna(subset=['src_airport.iata', 'dst_airport.iata', 'airline.iata'])
routes_df.isna().sum()

codeshare                     0
equipment                     0
airline.active                0
airline.airline_id            0
airline.alias                 0
airline.callsign              0
airline.country               0
airline.iata                  0
airline.icao                  0
airline.name                  0
dst_airport.airport_id        0
dst_airport.altitude          0
dst_airport.city              0
dst_airport.country           0
dst_airport.dst               0
dst_airport.iata              0
dst_airport.icao              0
dst_airport.latitude          0
dst_airport.longitude         0
dst_airport.name              0
dst_airport.source            0
dst_airport.timezone          0
dst_airport.type              0
dst_airport.tz_id             0
src_airport.airport_id        0
src_airport.altitude          0
src_airport.city              0
src_airport.country           0
src_airport.dst               0
src_airport.iata              0
src_airport.icao              0
src_airp

In [5]:
def key_gen(row):
    return str(row['src_airport.iata']) + str(row['dst_airport.iata']) + str(row['airline.iata'])

In [6]:
routes_df["key"] = routes_df.apply(key_gen, axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key
66766,CTU,NKG,ZH,CTUNKGZH
16546,KWL,CKG,CA,KWLCKGCA
16142,CKG,KMG,CA,CKGKMGCA
36691,ILM,ATL,KL,ILMATLKL
5555,IAH,ORD,AA,IAHORDAA
24864,ATL,CUN,FL,ATLCUNFL
59322,GSO,PHL,US,GSOPHLUS
8710,MAO,TFF,AD,MAOTFFAD
3952,AUH,TRV,9W,AUHTRV9W
56604,IAD,BOG,UA,IADBOGUA


In [7]:
partitions = (
    ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
    ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
    ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
    ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
)

def kv_key_gen(row):
    for tuple in partitions:
        first_char = row["key"][:1]
        if first_char >= tuple[0] and first_char <= tuple[1]:
            if tuple[0] == tuple[1]:
                return tuple[0]
            else:
                return tuple[0] + '-' + tuple[1]

In [8]:
routes_df["kv_key"] = routes_df.apply(kv_key_gen, axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key', 'kv_key']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key,kv_key
13295,FCO,FRA,AZ,FCOFRAAZ,E-F
15622,GUM,TPE,BR,GUMTPEBR,G-H
26978,HHN,TMP,FR,HHNTMPFR,G-H
44848,SPU,CDG,OU,SPUCDGOU,S-T
43325,SRA,GEL,nan,SRAGELnan,S-T
39960,OZH,VKO,M9,OZHVKOM9,O-P
33157,LIN,NAP,IG,LINNAPIG,K-L
23960,CTU,CAN,EU,CTUCANEU,C-D
54917,LGW,ALC,U2,LGWALCU2,K-L
24914,BDL,ATL,FL,BDLATLFL,B


In [9]:
routes_df.to_parquet('results/kv/', partition_cols=['kv_key'])

### Part b

In [10]:
import hashlib

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

In [11]:
routes_df["hashed"] = routes_df.apply(lambda x: hash_key(x["key"]), axis=1)
routes_df[['src_airport.iata', 'dst_airport.iata', 'airline.iata', 'key', 'hashed']].sample(10)

,src_airport.iata,dst_airport.iata,airline.iata,key,hashed
3054,WLK,OTZ,8E,WLKOTZ8E,04154d9573f0af18b1c17858d6d0337e7e9926e20be767...
42837,FUK,FUJ,NH,FUKFUJNH,4a84f0ec368ac45d25f43b443b82b5de7eae34dbc719e2...
18104,CAN,AVA,CZ,CANAVACZ,4ee8d3847d5766ed4b9c931eec9cb31e1a48e8ca22f37a...
57993,JED,HBE,UJ,JEDHBEUJ,00dd8987322b2b1db7e8806e199457bdc7461c8d65a6f9...
6558,PHX,MSP,AA,PHXMSPAA,6370b2725731c9b6b3425e80aab7650563e46d590f740f...
37910,LUX,FAO,LG,LUXFAOLG,3721338429aa3b26ed787c62ad5d3e14ba635640a49996...
46589,AUU,CNS,Q6,AUUCNSQ6,0c9ec1de06445f3613f75582e3434189da6a9be463b6c0...
15211,LBA,GLA,BE,LBAGLABE,bacfa6ecd470a65363de0f75c99700b3fde776dedfdf26...
3181,WUH,LJG,8L,WUHLJG8L,149d47f0899102a4d9957d454f389ff6f7acfb62af74b0...
42386,TSN,WUH,MU,TSNWUHMU,c8943f873f0588e4bb4292acb5fc47a4ba994dbd6117ef...


In [12]:
routes_df["hash_key"] = routes_df.apply(lambda x: x["hashed"][0].upper(), axis=1)

In [13]:
routes_df.to_parquet('results/hash/', partition_cols=['hash_key'])

### Part c

In [14]:
import pygeohash

In [15]:
data_centers = {'Location': ['West', 'Central', 'East'],
        'City': ['The Dalles, Oregon', 'Papillion, NE', 'Loudoun County, Virginia'],
        'Latitude': [45.5945645, 41.1544433, 39.08344],
        'Longitude': [-121.1786823, -96.0422378, -77.6497145]}

data_center_df = pd.DataFrame(data_centers)
data_center_df


,Location,City,Latitude,Longitude
0,West,"The Dalles, Oregon",45.594564,-121.178682
1,Central,"Papillion, NE",41.154443,-96.042238
2,East,"Loudoun County, Virginia",39.083440,-77.649715


In [16]:
# Assignment 7.1.c
routes_df['src_airport_geohash'] = routes_df.apply(
    lambda row: pygeohash.encode(row['src_airport.latitude'], row['src_airport.longitude']), axis=1
)
def determine_location(src_airport_geohash):
    locations = dict(
        central=pygeohash.encode(41.1544433, -96.0422378),
        east=pygeohash.encode(39.083440, -77.649715),
        west=pygeohash.encode(45.594564, -121.178682)
        ## TODO: add west and east
    )
    
    distances = [[pygeohash.geohash_haversine_distance(locations[center], src_airport_geohash), center] for center in locations]
    
    distances.sort()
    return distances[0][1]
routes_df['location'] = routes_df['src_airport_geohash'].apply(determine_location)
routes_df.to_parquet('results/geo', partition_cols=['location'])

In [17]:
routes_df[['src_airport.iata', 'src_airport.latitude', 'src_airport.longitude', 'key', 'location']].sample(10)

,src_airport.iata,src_airport.latitude,src_airport.longitude,key,location
39363,LBA,53.865898,-1.660570,LBABUDLS,east
56581,HNL,21.320620,-157.924228,HNLOGGUA,west
21478,MTY,25.778500,-100.107002,MTYLASDL,central
60990,BME,-17.944700,122.232002,BMEPERVA,west
59405,JAN,32.311199,-90.075897,JANDFWUS,central
63790,FLL,26.072599,-80.152702,FLLBNAWN,east
56336,EWR,40.692501,-74.168701,EWRINDUA,east
579,TIV,42.404701,18.723301,TIVDME3R,east
26276,BUD,47.429760,19.261093,BUDBRSFR,east
27462,NYO,58.788601,16.912201,NYOWMIFR,east


In [18]:
routes_df.to_parquet('results/geo', partition_cols=['location'])

### Part d

In [19]:
def balance_partitions(keys, num_partitions):
    partition_size = (len(keys) + num_partitions - 1) // num_partitions
    partitions = [keys[i:i + partition_size] for i in range(0, len(keys), partition_size)]
    partitions = [sorted(partition) for partition in partitions]
    return partitions

In [20]:
balanced_example = balance_partitions(list(routes_df.key), 30)
print(len(balanced_example))

for partition in balanced_example:
    print(len(partition))

30
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2226
2217
